# Finite Metropolis algorithm

#### Josep Fortiana 2018-11-21

### Metropolis algorithm for a finite state space

Observations $x$ follow a binomial $\operatorname{B}(n,\theta)$ distribution. Its support is the set of integers 

$$
    \left\{0,1,\dots,n\right\}.
$$

### Prior distribution of $\theta$

A discrete uniform distribution with support on the set:

$$
	I_m=\left\{0,\,\frac{1}{m},\,\frac{2}{m},\,\dots,\,\frac{m-1}{m},\,\frac{m}{m}=1\right\},	
$$										

with $(m+1)$ equally probable points, each with probability mass $\dfrac{1}{m+1}$

In [1]:
m<-10  # A small value, to be able to visualize everything
theta.values<-(0:m)/m #~ possible values of the probability parameter in the likelihood
prior<-rep(1/(m+1),m+1)

###	Likelihood

A function $\cal{L}(x\,|\,\theta)$ of the observation(s) $x$, probability mass function of $x$, given $\theta$.

By default, the function `Lik` is written so that $x$ is a scalar (not a vector) and the returned object is a vector, with the $(m+1)$ probabilities of $x$  values, given all possible $\theta$ values. 

`Liks` is a matrix with $(n+1)$ rows and $(m+1)$ columns. One row for each possible $x$, $0$ to $n$. One column for each of the $m+1$ possible $\theta$ values, $0$ to $1$, by increments of $1/m$.

Each column contains the values of the pmf, given the corresponding $\theta$ value.

Thus, `Liks` is a matrix of column profiles (in the notation given in the chapter on bivariate distributions); each column is a probability vector, adding up to 1.

In [2]:
n<-8   # A small value, to be able to visualize everything
x.values<-0:n
Lik<-function(x,theta=theta.values){dbinom(x,size=n,prob=theta)}
Liks<-t(sapply(x.values,Lik))
round(Liks,3)

1,0.430,0.168,0.058,0.017,0.004,0.001,0.000,0.000,0.000,0
0,0.383,0.336,0.198,0.090,0.031,0.008,0.001,0.000,0.000,0
0,0.149,0.294,0.296,0.209,0.109,0.041,0.010,0.001,0.000,0
0,0.033,0.147,0.254,0.279,0.219,0.124,0.047,0.009,0.000,0
0,0.005,0.046,0.136,0.232,0.273,0.232,0.136,0.046,0.005,0
0,0.000,0.009,0.047,0.124,0.219,0.279,0.254,0.147,0.033,0
0,0.000,0.001,0.010,0.041,0.109,0.209,0.296,0.294,0.149,0
0,0.000,0.000,0.001,0.008,0.031,0.090,0.198,0.336,0.383,0
0,0.000,0.000,0.000,0.001,0.004,0.017,0.058,0.168,0.430,1


Check that columns add up to 1

In [3]:
apply(Liks,2,sum)

[1] 1 1 1 1 1 1 1 1 1 1 1

### Joint distribution of the pair $(x,\theta)$

`Joint` is a matrix with $(n+1)$ rows and $(m+1)$ columns, containing the joint pmf of $(x,\theta)$

$$
\texttt{Joint[i,j]}=\operatorname{Prob}(x=\texttt{x.values[i]},\theta=\texttt{theta.values[j]})
$$

`Joint` can be computed either as a matrix product, by multiplying each column by the corresponding prior probability: 

```
Joint<-Liks %*% diag(prior)
```

or, taking advantage of the product vectorization in R, as follows:

In [4]:
Joint<-Liks*prior
round(Joint,3)

0.091,0.039,0.015,0.005,0.002,0.000,0.000,0.000,0.000,0.000,0.000
0.000,0.035,0.031,0.018,0.008,0.003,0.001,0.000,0.000,0.000,0.000
0.000,0.014,0.027,0.027,0.019,0.010,0.004,0.001,0.000,0.000,0.000
0.000,0.003,0.013,0.023,0.025,0.020,0.011,0.004,0.001,0.000,0.000
0.000,0.000,0.004,0.012,0.021,0.025,0.021,0.012,0.004,0.000,0.000
0.000,0.000,0.001,0.004,0.011,0.020,0.025,0.023,0.013,0.003,0.000
0.000,0.000,0.000,0.001,0.004,0.010,0.019,0.027,0.027,0.014,0.000
0.000,0.000,0.000,0.000,0.001,0.003,0.008,0.018,0.031,0.035,0.000
0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.005,0.015,0.039,0.091


### Marginal distribution of $x$

In this simple example we can compute explicitly the marginal distribution of $x$ (that is, the prior predictive distribution), and, given an observed $x$, $x=x_0$, say, the posterior distribution of $\theta$.

We do this, but later we will undertake a Metropolis MCMC simulation of this posterior distribution and compare both results.

`Marginal.x` is a vector of length $(n+1)$. 

MCMC simulation requires only the posterior numerator, not this marginal of $x$, which appears in the denominator.

In [5]:
Marginal.x<-apply(Joint,1,sum)
round(Marginal.x,3)

[1] 0.152 0.095 0.101 0.101 0.101 0.101 0.101 0.095 0.152

### Posterior distribution of $\theta$

In a real problem we would compute the posterior pmf of theta for the observed $x_0$.

Since this is a toy example, we are able to obtain a whole collection of posterior pmf's, for all possible values of $x$. The result is `Posteriors`, an $(n+1)\times(m+1)$ matrix, where each row contains the posterior pmf of theta, given the corresponding $x$.

Algebraically, `Posteriors` is the row profiles matrix of `Joint`:

Each $i$-th row contains the pmf of theta, conditioned to the corresponding $i$-th $x$ value, that is, the posterior distribution of theta assuming we observed $x=\texttt{x.values[i]}$. Thus, each row is a probability vector, with sum equal to 1.

In [6]:
Posteriors<-diag(1/Marginal.x)%*%Joint
round(Posteriors,3)

0.596,0.257,0.100,0.034,0.010,0.002,0.000,0.000,0.000,0.000,0.000
0.000,0.366,0.321,0.189,0.086,0.030,0.008,0.001,0.000,0.000,0.000
0.000,0.134,0.265,0.267,0.188,0.099,0.037,0.009,0.001,0.000,0.000
0.000,0.030,0.132,0.229,0.251,0.197,0.111,0.042,0.008,0.000,0.000
0.000,0.004,0.041,0.123,0.209,0.246,0.209,0.123,0.041,0.004,0.000
0.000,0.000,0.008,0.042,0.111,0.197,0.251,0.229,0.132,0.030,0.000
0.000,0.000,0.001,0.009,0.037,0.099,0.188,0.267,0.265,0.134,0.000
0.000,0.000,0.000,0.001,0.008,0.030,0.086,0.189,0.321,0.366,0.000
0.000,0.000,0.000,0.000,0.000,0.002,0.010,0.034,0.100,0.257,0.596


Check that rows of `Posteriors` add up to 1

In [7]:
apply(Posteriors,1,sum)

[1] 1 1 1 1 1 1 1 1 1

### Non-normalized posterior (for Metropolis algorithm)

Now, once we have the whole set of possible posterior pdf's, we will concentrate on a realistic MCMC simulation scenario, with a single posterior, associated with an $x$ observation. 

Henceforth we fix a value $x=x_0$, which we take as the observed value of $x$.

Since the $x$ marginal, the denominator which will get cancelled when computing the Metropolis probabilities of transition is not needed for the simulation, we omit it and obtain, for $x=x_0$, the non-normalized posterior.

The result, `PosteriorNUM` is a vector with $(m+1)$ entries, the $x=x_0$ row of `Joint`, times the prior pdf of $\theta$.

In [8]:
x0<-4
posteriorNUM<-Joint[x0+1,] # rows start at x=0
round(posteriorNUM,3)

[1] 0.000 0.000 0.004 0.012 0.021 0.025 0.021 0.012 0.004 0.000 0.000

In [9]:
round(sum(posteriorNUM),3)

[1] 0.101

Note that `sum(posteriorNUM)` is not 1; `posteriorNUM` is NOT a pmf, it is just _proportional_ to a pmf, which is enough for the simulation.

However, should we be able to normalize it, which is easily done in this toy example, but not in general in MCMC simulation real situations, either discrete or continuous, obvioulsy we would obtain a pmf. 

Actually this posterior pmf is the target of MCMC simulation: we design a Markov chain such that its limiting stationary distribution is this posterior and then, using its transition matrix, we generate trajectories of a given length $L$ and study the frequencies of their ending points, which we expect will be good approximations of their posterior  probabilities.

Here we evaluate it for comparison purposes. Check that the result is row $x_0+1=5$ in `Posteriors`.

In [10]:
posterior<-posteriorNUM/sum(posteriorNUM)
round(posterior,3)

[1] 0.000 0.004 0.041 0.123 0.209 0.246 0.209 0.123 0.041 0.004 0.000

## MCMC simulation

### Candidates

$K$ must be a square, symmetric, stochastic $(m+1)\times (m+1)$ matrix (as many rows and columns as $\theta$ states).

Each row contains the transition probabilities from the current state within $\texttt{theta.values}$ to a new, proposed _candidate_ state. 

Then we will either accept or reject this candidate

Here we take for $K$ the transition matrix of a symmetric cyclic random walk. That is, in a given time, we can either proceed to the right or to the left with equal probability, except when the current position is an extreme, right or left, where there is 50% probability to go to the opposite one.

In [11]:
K<-matrix(rep(0,(m+1)*(m+1)),nrow=m+1)
K[1,2]<-0.5;K[1,m+1]<-0.5
K[m+1,m]<-0.5;K[m+1,1]<-0.5
for (j in 2:m){
    K[j,j-1]<-0.5
    K[j,j+1]<-0.5
    }
round(K,2)

0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5
0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0


###	Acceptance probabilities

Each transition is acted upon by an acceptance probability. The matrix $U$ with these acceptance probabilities is an $(m+1)\times (m+1)$ matrix, with entries:

$$
	U[\theta_1,\theta_2]=\min\left\{1,
    \dfrac{\texttt{posteriorNUM}[\theta_2]}{\texttt{posteriorNUM}[\theta_1]}\right\}
$$

A little something to avoid NaN's: entries in $U$ smaller than a given $\varepsilon$ are set to 1 before putting them as the denominator in this formula. Finally we set all diagonal entries in $U$ to 0. 

Later on diagonal entries in the transition matrix, obtained by Hadamard (elementwise) product of $K$ and $U$, will be set to 1 minus the sum of the remaining entries in the row, to form a stochastic matrix, see below.

In [12]:
#~ epsilor is not important here, just to avoid infinities or nans or so on.
epsilon<-1.0e-5
posteriorNUM.1<-ifelse(posteriorNUM<epsilon,1,posteriorNUM)
ones<-matrix(1,nrow=m+1,ncol=m+1)
#~ str(posteriorNUM.1)
#~ str(posteriorNUM)
U<-pmin(ones,(1/posteriorNUM.1)%*%t(posteriorNUM)) #~ The matrix version containing the coefficients of the probabilities Pij (يلي كتبنا عليها ملاحظة انها اليمينت وايز)
diag(U)<-0
round(U,3)

0,0.000,0.004,0.012,0.021,0.025,0.021,0.012,0.004,0.000,0
0,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0
0,0.100,0.000,1.000,1.000,1.000,1.000,1.000,1.000,0.100,0
0,0.034,0.337,0.000,1.000,1.000,1.000,1.000,0.337,0.034,0
0,0.020,0.198,0.586,0.000,1.000,1.000,0.586,0.198,0.020,0
0,0.017,0.168,0.498,0.849,0.000,0.849,0.498,0.168,0.017,0
0,0.020,0.198,0.586,1.000,1.000,0.000,0.586,0.198,0.020,0
0,0.034,0.337,1.000,1.000,1.000,1.000,0.000,0.337,0.034,0
0,0.100,1.000,1.000,1.000,1.000,1.000,1.000,0.000,0.100,0
0,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000,0
0,0.000,0.004,0.012,0.021,0.025,0.021,0.012,0.004,0.000,0


#### Transition matrix

To prepare the transition matrix, according to the Metropolis algorithm, we need to multiply elementwise: 

$$
    P_0[i,j]=K[i,j]\cdot U[i,j]
$$

In matrix notation, $P_0=K \cdot U$, this is a Hadamard, not matrix, product. 

Finally set diagonal entries in $P_0$ (which currently are zero) to the values needed so that the resulting $P$ is a stochastic matrix.

In [13]:
#~ starting from a non-normalized posterior and a random walk, I obtained...
P0<-K*U
round(P0,3)
P0.rows.sum<-apply(P0,1,sum) 
round(P0.rows.sum,3)
P<-P0+diag(1-P0.rows.sum)
round(P,3)

0,0.00,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.00,0
0,0.00,0.500,0.000,0.000,0.0,0.000,0.000,0.000,0.00,0
0,0.05,0.000,0.500,0.000,0.0,0.000,0.000,0.000,0.00,0
0,0.00,0.168,0.000,0.500,0.0,0.000,0.000,0.000,0.00,0
0,0.00,0.000,0.293,0.000,0.5,0.000,0.000,0.000,0.00,0
0,0.00,0.000,0.000,0.425,0.0,0.425,0.000,0.000,0.00,0
0,0.00,0.000,0.000,0.000,0.5,0.000,0.293,0.000,0.00,0
0,0.00,0.000,0.000,0.000,0.0,0.500,0.000,0.168,0.00,0
0,0.00,0.000,0.000,0.000,0.0,0.000,0.500,0.000,0.05,0
0,0.00,0.000,0.000,0.000,0.0,0.000,0.000,0.500,0.00,0
0,0.00,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.00,0


[1] 0.000 0.500 0.550 0.668 0.793 0.849 0.793 0.668 0.550 0.500 0.000

1,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0
0,0.50,0.500,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0
0,0.05,0.450,0.500,0.000,0.000,0.000,0.000,0.000,0.00,0
0,0.00,0.168,0.332,0.500,0.000,0.000,0.000,0.000,0.00,0
0,0.00,0.000,0.293,0.207,0.500,0.000,0.000,0.000,0.00,0
0,0.00,0.000,0.000,0.425,0.151,0.425,0.000,0.000,0.00,0
0,0.00,0.000,0.000,0.000,0.500,0.207,0.293,0.000,0.00,0
0,0.00,0.000,0.000,0.000,0.000,0.500,0.332,0.168,0.00,0
0,0.00,0.000,0.000,0.000,0.000,0.000,0.500,0.450,0.05,0
0,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.500,0.50,0
0,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,1


Check that indeed this transition matrix does define a Markov chain whose stationary distribution is the posterior distribution. To do this, 

1. Compute the eigen decomposition: $P = V^{-1} \cdot D \cdot V$. The `eigen()` function returns $V$.

2. Compute $V_1 = V^{-1}$.

3. The stationary distribution will be the first row in $V_1$, after being standardized to sum $1$.

In [14]:
a<-eigen(P)
V<-a$vectors
V1<-solve(V)
v<-V1[1,]
w<-v/sum(v)
round(w,3)
# (compare with the posterior probability above)
round(posterior,3)

[1] 0.000 0.004 0.041 0.123 0.209 0.246 0.209 0.123 0.041 0.004 0.000

[1] 0.000 0.004 0.041 0.123 0.209 0.246 0.209 0.123 0.041 0.004 0.000

#### Prepare the Metropolis MCMC simulation

We prepare some auxiliary functions 

In [15]:
#
# r.idx 
#
# Auxiliary function.
#
# Generate a random integer r in the set 1:m according to the probability vector d. 
#
# Then, if we want a random vector from a vector x such that length(x)=m, we just return x[r]
# We assume (though no formal check is performed) that d is a well-formed probability vector.
#
# --------------------------------------------------------------------------------------------------
r.idx<-function(d){
    m<-length(d)
    return(sample.int(m,size=1,prob=d))
    }

In [16]:
#
# one.trajectory
#
# Auxiliary function.
#
# Generate a single trajectory of length L of the Markov chain with transition probabilities 
# matrix P and initial probability p. 
#
# Only indexes in the vector x of values are obtained. 
#
# --------------------------------------------------------------------------------------------------
one.trajectory<-function(L=100,P,p){
    idx.j<-r.idx(p)
    trajectory<-rep(0,L) 
    for (j in 1:L){
        idx.j<-r.idx(P[idx.j,])
        trajectory[j]<-idx.j
        }
    return(trajectory)
    }

In [21]:
#
# N.trajectories
#
# Auxiliary function.
#
# Generate N length L trajectories of the Markov chain with transition probabilities matrix P 
# and initial probability p. 
#
# Returns an [N,L] matrix. Each row is a trajectory.
#
# Only indexes in the vector x of values are obtained. 
#
# --------------------------------------------------------------------------------------------------
N.trajectories<-function(N=500,L=100,P,p){
    trajectories<-matrix(0,nrow=N,ncol=L)
    for (i in 1:N){
        trajectories[i,]<-one.trajectory(L,P,p)
        }
    return(trajectories)
    }

In [22]:
#
# limit.prob
#
# Auxiliary function.
#
# Estimate the limit probability of the Markov chain with transition probabilities matrix P and 
# initial probability p. To this end we generate N trajectories of length L and tabulate their
# end points.
#
# --------------------------------------------------------------------------------------------------
limit.prob<-function(N,L,P,p){
    trajectories<-N.trajectories(N,L,P,p)
    prob<-tabulate(trajectories[,L])/N
    return(prob)
    }

In [19]:
#
# limit.prob.k
#
# Auxiliary function.
#
# (modification of the above)
# Estimate the limit probability of the Markov chain with transition probabilities matrix P and 
# initial probability p. 
# To this end we generate N trajectories of length L and tabulate their end segments of length k.
#
# --------------------------------------------------------------------------------------------------
limit.prob.k<-function(N,L,P,p,k){
    trajectories<-N.trajectories(N,L,P,p)
    Z<-trajectories[,(L-k+1):L]
    prob<-tabulate(as.numeric(Z))/(N*k)
    return(prob)
    }

### Now launch the Metropolis MCMC simulation

In [23]:
#p<-limit.prob(500,1000,P,prior);round(p,3)
p<-limit.prob.k(100,10000,P,prior,1000);round(p,3)

[1] 0.001 0.004 0.040 0.121 0.207 0.244 0.207 0.122 0.040 0.004 0.010

#### Chi square test

Since for this toy example we know the exact posterior pdf, we can test the goodness-of-fit of the simulated Markov chain limit to the exact pdf.

In [24]:
#
# chisq1d
#
# Auxiliary function.
#
# One dimensional chi-square test to compare a probability (or absolute frequencies) vector nn
# to a probability vector p, taken as reference.
#
# --------------------------------------------------------------------------------------------------
chisq1d<-function(nn,p,df=length(nn)-1,alpha=0.05){
    n<-sum(nn)
    ff<-nn/n
    chi2<-n*sum(((ff-p)^2)/p)
    pval=1-pchisq(chi2,df=df)
    valcrit=qchisq(1-alpha,df=df)
    return(list(chi2=chi2,df=df,pval=pval,valcrit=valcrit))
    }

In [25]:
# Applying the chi-square test. Ignore 0 values in the posterior vector, as they would give NaN in the formula.
J<-posterior==0
chisq1d(p[!J],posterior[!J])

$chi2
[1] 6.269855e-05

$df
[1] 8

$pval
[1] 1

$valcrit
[1] 15.50731

### Exercise

Redo the above computations with a prior pdf $\operatorname{Beta}(\alpha,\beta)$, for $(\alpha,\beta)\neq(1,1)$, discretized on a grid of $m$ (or $m+1$) points. _Hint:_ Be careful with the number of points and intervals.

In [ ]:
#~ just for you to check that you have understood all the steps in ...